In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [2]:
df=pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.shape

(768, 9)

In [5]:
# correlation of Outcome columns with respects to other colmns
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
#check for null,duplicated and nan value
df.isnull().sum(),df.duplicated().sum(),df.isna().sum()

(Pregnancies                 0
 Glucose                     0
 BloodPressure               0
 SkinThickness               0
 Insulin                     0
 BMI                         0
 DiabetesPedigreeFunction    0
 Age                         0
 Outcome                     0
 dtype: int64,
 0,
 Pregnancies                 0
 Glucose                     0
 BloodPressure               0
 SkinThickness               0
 Insulin                     0
 BMI                         0
 DiabetesPedigreeFunction    0
 Age                         0
 Outcome                     0
 dtype: int64)

In [7]:
#separating dependent and independent value
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [8]:
#train_test_split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [9]:
#Scaling the train and test data 
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [10]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

2025-10-17 14:39:12.302858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760711952.517407      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760711952.575703      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
#creating the neural network model
tf.random.set_seed(43)
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=(X.shape[1])))
model.add(Dense(1,activation='sigmoid'))
#compile the model
loss=tf.keras.losses.BinaryCrossentropy()
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=['accuracy']
model.compile(loss=loss,optimizer=optimizer,metrics=metrics)
#callbacks 
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='accuracy',patience=5,restore_best_weights=True)
#fit the model
model.fit(X_train_scaled,y_train,epochs=100,callbacks=[early_stopping],validation_data=(X_test_scaled,y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1760711963.855842      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1760711963.856511      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/100


I0000 00:00:1760711966.289059      60 service.cc:148] XLA service 0x7bcb7c006cd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760711966.289388      60 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1760711966.289434      60 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1760711966.476911      60 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/18 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.4375 - loss: 0.7004

I0000 00:00:1760711966.996906      60 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.4780 - loss: 0.6967 - val_accuracy: 0.6510 - val_loss: 0.6796
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6568 - loss: 0.6747 - val_accuracy: 0.6354 - val_loss: 0.6685
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6595 - loss: 0.6627 - val_accuracy: 0.6406 - val_loss: 0.6620
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6595 - loss: 0.6548 - val_accuracy: 0.6406 - val_loss: 0.6566
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6595 - loss: 0.6479 - val_accuracy: 0.6406 - val_loss: 0.6508
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6591 - loss: 0.6416 - val_accuracy: 0.6458 - val_loss: 0.6456
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6591 - loss: 0.6365 - val_accuracy: 0.6458 - val_loss: 0.6410


# hyperparameter tuning for optimizers

In [12]:
import keras_tuner as kt
def build_model(hp):
    model1=Sequential()
    model1.add(Dense(32,activation='relu',input_dim=(X.shape[1])))
    model1.add(Dense(1,activation='sigmoid'))
    #compile the model
    optimizer=hp.Choice('otimizers',values=['adam','sgd','rmsprop','adadelta'])
    loss=tf.keras.losses.BinaryCrossentropy()
    metrics=['accuracy']
    model1.compile(optimizer=optimizer,loss=loss,metrics=metrics)
    return model1

In [13]:
tuner1=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [14]:
tuner1.search(X_train_scaled,y_train,epochs=100,callbacks=[early_stopping],validation_data=(X_test_scaled,y_test))

Trial 4 Complete [00h 00m 07s]
val_accuracy: 0.703125

Best val_accuracy So Far: 0.703125
Total elapsed time: 00h 00m 20s


In [15]:
tuner1.get_best_hyperparameters()[0].values

{'otimizers': 'adam'}

In [16]:
#get best model
model2=tuner1.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [17]:
model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
#fit the model
model2.fit(X_train_scaled,y_train,epochs=100,callbacks=[early_stopping],validation_data=(X_test_scaled,y_test))

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7661 - loss: 0.4982 - val_accuracy: 0.7031 - val_loss: 0.5506
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7686 - loss: 0.4952 - val_accuracy: 0.7031 - val_loss: 0.5493
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7686 - loss: 0.4935 - val_accuracy: 0.7031 - val_loss: 0.5483
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7689 - loss: 0.4919 - val_accuracy: 0.7031 - val_loss: 0.5473
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7669 - loss: 0.4903 - val_accuracy: 0.6979 - val_loss: 0.5463
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7684 - loss: 0.4887 - val_accuracy: 0.6979 - val_loss: 0.5454
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7773 - loss: 0.4871 - val_accuracy: 0.7031 - val_loss: 0.5445
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7797 - loss: 0.4856 - val_accuracy: 0.7083 - 

# Hyperparameters tuning for Neurons

In [19]:
def build_model1(hp):
    model3=Sequential()
    units=hp.Int('units',min_value=8,max_value=128,step=8)
    model3.add(Dense(units=units,activation='relu',input_dim=(X_train.shape[1])))
    model3.add(Dense(1,activation='sigmoid'))
    #compile the model
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
    model3.compile(loss=loss,optimizer=optimizer,metrics=metrics)
    return model3

In [20]:
tuner3=kt.RandomSearch(build_model1,objective='val_accuracy',max_trials=5,directory='neuron_dir',project_name='neuron_1')

In [21]:
tuner3.search(X_train_scaled,y_train,epochs=10,validation_data=(X_test_scaled,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6614583134651184

Best val_accuracy So Far: 0.6666666865348816
Total elapsed time: 00h 00m 17s


In [22]:
tuner3.get_best_hyperparameters()[0].values

{'units': 64}

In [23]:
#get the best model
model3_best=tuner3.get_best_models(num_models=1)[0]

In [24]:
model3_best.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 641 (2.50 KB)

 Trainable params: 641 (2.50 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model3_best.fit(X_train_scaled,y_train,epochs=100,initial_epoch=11,callbacks=[early_stopping],validation_data=(X_test_scaled,y_test))

Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.6595 - loss: 0.6695 - val_accuracy: 0.6406 - val_loss: 0.6624
Epoch 13/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6595 - loss: 0.6548 - val_accuracy: 0.6406 - val_loss: 0.6572
Epoch 14/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6595 - loss: 0.6480 - val_accuracy: 0.6406 - val_loss: 0.6525
Epoch 15/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6595 - loss: 0.6417 - val_accuracy: 0.6458 - val_loss: 0.6472
Epoch 16/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6610 - loss: 0.6357 - val_accuracy: 0.6458 - val_loss: 0.6419
Epoch 17/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6624 - loss: 0.6297 - val_accuracy: 0.6458 - val_loss: 0.6369
Epoch 18/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6679 - loss: 0.6236 - val_accuracy: 0.6458 - val_loss: 0.6317
Epoch 19/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6746 - loss: 0.6174 - val_accuracy: 0